# CHAMP Mutation list

This code read the CHAMP Mutation List excel file from CDC and clean it to be easier to manipulate and with less errors. 

In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
# read sheet from excel that has list off mutations from hemophilia A
file="champ-mutation-list-q4.xlsx"
sheet="CHAMP Mutation List"
df = pd.read_excel(file, sheet)

In [3]:
# clean the columns to increase readability
df.columns = [c.replace('\n', ' ').replace('  ',' ').strip() for c in df.columns]

In [4]:
# trim all values that are not NaN to facilitate manipulation
df = df.map(lambda x: str(x).strip() if x is not np.nan else x)

In [5]:
# function to apply regex search and return NaN if dont find it
def re_search(regex, row):
    r = re.search(regex, row)
    return np.nan if r is None else str(r.group()[2:])

# functions to split the HGVS Protein column
def get_wild_aa(s):
    return str(re.search("^\D+", s).group())

def get_pos_mut(s):
    return int(re.search("\d+", s).group())

def get_new_aa(s):
    return str(re.search("\D+$", s).group())

In [6]:
# define invalid values as NaN from HGVS Protein column 
regex = "(^p.[A-Z]{1}[a-z]{2}\d*(\*{1}|[A-Z]{1}[a-z]{2})$)"
df["HGVS Protein"] = df["HGVS Protein"].apply(lambda s: re_search(regex, str(s)))

In [7]:
# define invalid values as NaN from Reported Severity column
df["Reported Severity"] = df["Reported Severity"].apply(lambda s: np.nan if '/' in s else s)
# define invalid values as NaN from Codon column
df["Codon"] = df["Codon"].apply(lambda s: np.nan if '-' in str(s) else s)

In [8]:
# list of columns filtered to use dropna method
cols = list(df.columns)
cols_remove = ['Severe (<1 U/dL)', 'Moderate (1-5 U/dL)', 'Mild (>5 U/dL)', 'Unclassified (no FVIII level)', 'Comments']
for c in cols_remove:
    cols.remove(c)
# drop rows with NaN based on above column list
df.dropna(inplace=True, subset=cols)

In [9]:
# create three new columns to facilitate later manipulation
df["HGVS Wild Amino Acid"] = df["HGVS Protein"].apply(lambda s: get_wild_aa(s))
df["HGVS New Amino Acid"] = df["HGVS Protein"].apply(lambda s: get_new_aa(s))
df["HGVS Position"] = df["HGVS Protein"].apply(lambda s: get_pos_mut(s))

In [10]:
# list of columns filtered to use title method from string type
cols = list(df.columns)
cols_remove = ['HGVS Protein', 'Mature Protein', 'Domain']
for c in cols_remove:
    cols.remove(c)
# capitilize strings based on above column list
df[cols] = df[cols].applymap(lambda x: x.title() if isinstance(x, str) else x)

/var/folders/yb/40nc7cwj2qx2lrp1gvs874bm0000gp/T/ipykernel_9398/4034558844.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(lambda x: x.title() if isinstance(x, str) else x)


In [11]:
# sort dataframe using Codon column as reference
df['Codon'] = df['Codon'].astype('int')
df.sort_values(inplace=True, by='Codon')

In [12]:
# reset index because the dropped rows
df.reset_index(inplace=True, drop=True)

In [13]:
df.head()

,HGVS cDNA,hg19 Coordinates,HGVS Protein,Mature Protein,Mutation Type,Mechanism,Exon,Codon,Domain,Subtype,...,Mild (>5 U/dL),Unclassified (no FVIII level),Reported Severity,History of Inhibitor,Comments,Reference ID,Year Reported,HGVS Wild Amino Acid,HGVS New Amino Acid,HGVS Position
0,C.65G>C,154250763,Arg22Thr,Arg3Thr,Missense,Substitution,1,3,A1,Heavy Chain,...,NaN,NaN,Severe,No,NaN,129,2002,Arg,Thr,22
1,C.64A>G,154250764,Arg22Gly,Arg3Gly,Missense,Substitution,1,3,A1,Heavy Chain,...,NaN,NaN,Severe,Not Reported,NaN,187,2008,Arg,Gly,22
2,C.65G>T,154250763,Arg22Ile,Arg3Ile,Missense,Substitution,1,3,A1,Heavy Chain,...,X,NaN,Mild,No,NaN,H,H,Arg,Ile,22
3,C.67A>G,154250761,Arg23Gly,Arg4Gly,Missense,Substitution,1,4,A1,Heavy Chain,...,X,NaN,Mild,No,NaN,H,H,Arg,Gly,23
4,C.72C>G,154250756,Tyr24*,Tyr5*,Nonsense,Substitution,1,5,A1,Heavy Chain,...,NaN,NaN,Severe,Not Reported,NaN,260,2014,Tyr,*,24


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2063 entries, 0 to 2062
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   HGVS cDNA                      2063 non-null   object
 1   hg19 Coordinates               2063 non-null   object
 2   HGVS Protein                   2063 non-null   object
 3   Mature Protein                 2063 non-null   object
 4   Mutation Type                  2063 non-null   object
 5   Mechanism                      2063 non-null   object
 6   Exon                           2063 non-null   object
 7   Codon                          2063 non-null   int64 
 8   Domain                         2063 non-null   object
 9   Subtype                        2063 non-null   object
 10  In Poly A                      2063 non-null   object
 11  Severe (<1 U/dL)               803 non-null    object
 12  Moderate (1-5 U/dL)            329 non-null    object
 13  Mil

In [15]:
# write the cleaned dataframe in a excel file
df.to_excel(f"{file[:-5]}-clean.xlsx")